<h1><center> Aggregated Data Noise Creation <br><br> 
<center> Scikit Mobility and DB Noise<br><br> 
Wendy Shi</center></center>

# Preparation

In [1]:
#Preparation
import pandas as pd
import numpy as np

## Load Count Data

In [2]:
ori_mt = pd.read_csv("AZ_Data/Aggregate/AZ_ori_indi.csv")
ori_mt = ori_mt.drop(columns = 'Unnamed: 0')
ori_mt.head(3)

,04001,04003,04005,04007,04009,04012,04013,04015,04017,04019,04021,04023,04025,04027
0,0.393417,0.006658,0.010795,0.002831,0.001058,0.000249,0.366134,0.005071,0.051549,0.073389,0.053571,0.002520,0.018510,0.014248
1,0.000343,0.728874,0.001962,0.001312,0.001092,0.000000,0.099431,0.000895,0.001472,0.147742,0.010781,0.005249,0.000503,0.000343
2,0.004619,0.001075,0.798236,0.004241,0.000060,0.000448,0.132651,0.003146,0.003126,0.018319,0.007397,0.000388,0.023207,0.003086


In [3]:
column_names = ['S', 'E', 'I', 'R']
pop_2020 = pd.read_csv("../../Data/us_shapefile_county.csv")
pop_2020 = pop_2020[['GEO_ID','pop2020']]
pop_2020["GEO_ID"] = pop_2020["GEO_ID"].apply(lambda x: f"{int(x):05d}")

rename = {'GEO_ID':'GEOID'}
pop_2020 = pop_2020.rename(columns = rename)

print(pop_2020.shape)
pop_2020.tail(3)

(3109, 2)


,GEOID,pop2020
3106,06037,9989165.0
3107,46113,14309.0
3108,51515,6599.0


In [4]:
county_list = ori_mt.columns.tolist()
county_pop = pd.DataFrame()
county_pop['GEOID'] = county_list
county_pop.head(3)

,GEOID
0,04001
1,04003
2,04005


In [5]:
# Get list for the population
joined = pd.merge(county_pop, pop_2020, on = 'GEOID', how = 'left')
pop_list = joined['pop2020'].tolist()
print(f'Length of the population list: {len(pop_list)}')
joined.head()

Length of the population list: 14


,GEOID,pop2020
0,04001,65897.0
1,04003,125398.0
2,04005,145116.0
3,04007,53306.0
4,04009,38614.0


In [6]:
pop_list = joined['pop2020'].tolist()
pop_list[:5]

[65897.0, 125398.0, 145116.0, 53306.0, 38614.0]

In [7]:
ori_count = ori_mt.multiply(pop_list, axis=0)
ori_count.head(3)

,04001,04003,04005,04007,04009,04012,04013,04015,04017,04019,04021,04023,04025,04027
0,25925.008151,438.711984,711.369431,186.555096,69.701904,16.400448,24127.109041,334.159128,3396.942789,4836.082099,3530.196429,166.054536,1219.783319,938.925647
1,43.063556,91399.322295,246.077465,164.564304,136.880590,0.000000,12468.437535,112.272843,184.558098,18526.557117,1351.888071,658.257218,63.057350,43.063556
2,670.361833,156.032496,115836.791127,615.461511,8.668472,65.013540,19249.786787,456.539525,453.650034,2658.331409,1073.445781,56.345068,3367.701366,447.871053


# Differential Pricavy Noise

In [8]:
#from opendp.trans import make_base_laplace
import opendp.prelude as dp
dp.enable_features("contrib")

In [42]:
count_noise = ori_count.astype(int).copy()
DP_df = pd.DataFrame()

for i in range(14):
    
    if i % 10 == 0: 
        print(i)
        
    epsilon = 0.001 #Privacy budget
    sensitivity = pop_list[i] * 0.0001 # Define sensitivity for each county
    scale = sensitivity / epsilon
    
    # Create the Laplace mechanism
    laplace_mechanism = dp.m.make_laplace(
        dp.atom_domain(T=int), dp.absolute_distance(T=int), scale
    )
    
    def apply_dp_noise(count):
        return laplace_mechanism(count)
    
    new_df_row = count_noise.iloc[[i]].applymap(lambda x: max(laplace_mechanism(x), 0))
    DP_df = pd.concat([DP_df, new_df_row], axis = 0)

0
10


In [43]:
#Original
ori_count.head(3)

,04001,04003,04005,04007,04009,04012,04013,04015,04017,04019,04021,04023,04025,04027
0,25925.008151,438.711984,711.369431,186.555096,69.701904,16.400448,24127.109041,334.159128,3396.942789,4836.082099,3530.196429,166.054536,1219.783319,938.925647
1,43.063556,91399.322295,246.077465,164.564304,136.880590,0.000000,12468.437535,112.272843,184.558098,18526.557117,1351.888071,658.257218,63.057350,43.063556
2,670.361833,156.032496,115836.791127,615.461511,8.668472,65.013540,19249.786787,456.539525,453.650034,2658.331409,1073.445781,56.345068,3367.701366,447.871053


In [44]:
DP_df.head(3)

,04001,04003,04005,04007,04009,04012,04013,04015,04017,04019,04021,04023,04025,04027
0,28405,20667,5542,0,33442,704,13022,7479,3965,5258,13108,11761,1670,14932
1,0,89747,0,0,9730,5735,47453,0,2058,0,0,0,7317,0
2,39302,0,97017,0,17200,340,30348,21778,0,0,1561,5129,19862,0


In [45]:
#Turn count to weight
divisor = [DP_df.iloc[i].sum() for i in range(14)]
noise_mt = DP_df.div(divisor, axis=0)
noise_mt.head(3)

,04001,04003,04005,04007,04009,04012,04013,04015,04017,04019,04021,04023,04025,04027
0,0.177581,0.129205,0.034647,0.0,0.209071,0.004401,0.081410,0.046757,0.024788,0.032872,0.081948,0.073527,0.010440,0.093351
1,0.000000,0.553857,0.000000,0.0,0.060047,0.035392,0.292847,0.000000,0.012701,0.000000,0.000000,0.000000,0.045156,0.000000
2,0.169014,0.000000,0.417211,0.0,0.073967,0.001462,0.130508,0.093654,0.000000,0.000000,0.006713,0.022057,0.085414,0.000000


In [46]:
# Calculations
import statistics

#noise_mt_np = ori_mt.to_numpy()
noise_mt_np = noise_mt.to_numpy()
Pii_skmob = [noise_mt_np[i][i] for i in range(14)]


data = Pii_skmob 
mean_value = sum(data) / len(data)  # Mean
median_value = statistics.median(data)  # Median
min_value = min(data)  # Min
max_value = max(data)  # Max

# Output
print(f"Mean: {mean_value}")
print(f"Median: {median_value}")
print(f"Min: {min_value}")
print(f"Max: {max_value}")

Mean: 0.42967907652489984
Median: 0.4388857075286874
Min: 0.12397514504342876
Max: 0.755127042361337


In [47]:
noise_mt.to_csv('AZ_P001_agg.csv')